In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/BDSProject/image-transformation-degradation/

/content/drive/MyDrive/BDSProject/image-transformation-degradation


In [4]:
!ls

colab_evaluateImageNet.ipynb  evaluateImageNet.ipynb  README.md


In [5]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import vgg16, resnet152, alexnet
import torch

import os
import json
from PIL import Image
import numpy as np

In [6]:
args = {"batch_size": 16}
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda:0


In [21]:
class ImageNetDataset(Dataset):
    def __init__(self, image_dir, annotations_file, transformations = None, device = 'cpu'):
        self.image_dir = image_dir
        self.images = os.listdir(self.image_dir)
        self.annotations = json.load(open(annotations_file))
        self.transformations = transformations
        self.device = device

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        img = Image.open(img_path).convert('RGB')
        label = torch.tensor(int(self.annotations[self.images[idx].split('.')[0]])).to(self.device)
        if self.transformations:
            img = self.transformations(img)
        img = img.to(self.device)
        return img, label


preprocessing = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
])
imagenetValDataset = ImageNetDataset(image_dir = '../data/ImageNet/Images/val_10p/images', 
                                    annotations_file='../data/ImageNet/Images/val_10p/imagnet_classes.json', 
                                    transformations=preprocessing, 
                                    device = device)

imagenetValDataloader = torch.utils.data.DataLoader(imagenetValDataset, batch_size=args['batch_size'], shuffle=True)


In [22]:
def compute_accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].contiguous().view(-1).float().sum(0)
        res.append(correct_k)
    return res

In [29]:
model = alexnet(pretrained = True)
model.eval()
model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [30]:
from tqdm import tqdm
accuracy_top1 = 0
accuracy_top5 = 0
count = 0
for batch_idx, (img, label) in enumerate(tqdm(imagenetValDataloader)):
    out = model(img).detach()
    count += out.shape[0]
    top1, top5 = compute_accuracy(out, label, topk = (1, 5))
    accuracy_top1 += top1
    accuracy_top5 += top5
    if(batch_idx % 100 == 0):
        print("Evaluated {}/{}: Top 1 Accuracy: {:.3f} Top 5 Accuracy: {:.3f}".format(str(count), 
                                                         str(len(imagenetValDataset)), accuracy_top1*100.0/count, accuracy_top5*100.0/count))
        
print("Evaluated {}/{}: Top 1 Accuracy: {:.3f} Top 5 Accuracy: {:.3f}".format(str(count), 
                                                         str(len(imagenetValDataset)), accuracy_top1*100.0/count, accuracy_top5*100.0/count))
        
        

  0%|          | 1/313 [00:00<01:03,  4.91it/s]

Evaluated 16/5000: Top 1 Accuracy: 68.750 Top 5 Accuracy: 81.250


 33%|███▎      | 102/313 [00:21<00:44,  4.71it/s]

Evaluated 1616/5000: Top 1 Accuracy: 59.282 Top 5 Accuracy: 81.002


 64%|██████▍   | 201/313 [00:41<00:23,  4.69it/s]

Evaluated 3216/5000: Top 1 Accuracy: 58.489 Top 5 Accuracy: 80.877


 96%|█████████▋| 302/313 [01:03<00:02,  4.64it/s]

Evaluated 4816/5000: Top 1 Accuracy: 57.579 Top 5 Accuracy: 80.274


100%|██████████| 313/313 [01:05<00:00,  4.80it/s]

Evaluated 5000/5000: Top 1 Accuracy: 57.600 Top 5 Accuracy: 80.140


In [31]:
print("Evaluated {}/{}: Top 1 Accuracy: {:.3f} Top 5 Accuracy: {:.3f}".format(str(count), 
                                                         str(len(imagenetValDataset)), accuracy_top1*100.0/count, accuracy_top5*100.0/count))

Evaluated 5000/5000: Top 1 Accuracy: 57.600 Top 5 Accuracy: 80.140
